# Stock Strategy
### Based on 5 years of data obtained from yahoo finance

#### Sources

- https://github.com/kunalkini15/technical_indicators_lib
- https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py
- https://www.investopedia.com/

### To do

- create a function that modifies and renames moving average columns
- create a function that takes a df and any technical indicator as arugments and returns the modified df
- redo the api call for date ranges

_____

## Import modules

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime, date
import seaborn as sns

In [6]:
from technical_indicators.library import indicators

In [5]:
from ti_lib.ti_lib.indicators import (
    ADI, ATR, BollingerBands, CCI, CMF, CHO, CHV, DC, DPO, EMV, EMA, FI, KC, KST, MI, MED, MOM,
    MFI, MACD, NVI, OBV, PVT, PC, PO, ROC, ROCV, RSI, SMA, VLT, StochasticKAndD,
    Trix, TR, TSI, TYP, VHF, VO, WCL)

adi = ADI()
atr = ATR()
bb = BollingerBands()
cci = CCI()
cmf = CMF()
cho = CHO()
chv = CHV()
dc = DC()
dpo = DPO()
emv = EMV()
ema = EMA()
fi = FI()
kc = KC()
kst = KST()
mi = MI()
mem = MED()
mom = MOM()
mfi = MFI()
macd = MACD()
nvi = NVI()
obv = OBV()
pvt = PVT()
pc = PC()
po = PO()
roc = ROC()
rocv = ROCV()
rsi = RSI()
sma = SMA()
vlt = VLT()
stochastic_k_and_d = StochasticKAndD()
trix = Trix()
tr = TR()
tsi = TSI()
typ = TYP()
vhf = VHF()
vo = VO()
wcl = WCL()

## Get data from YahooFinance API

In [ ]:
def get_yf_data(ticker, start='2018-01-01', end=date.today()):
    '''
    Calls Yahoo Finance API and retrieves historical data for given ticker and time period
    
    Args:
        ticker(string): string, stock ticker
        start (string): start date, default 2018-01-01
        end (datetime object): end date, default today's date
        
    Returns:
        pandas.DataFrame: new pandas dataframe with the OHLC data for the given ticker and time period
    '''
    # get historical data
    temp = yf.Ticker(ticker)
    df = temp.history(start=start, end=end)
    # make columns lowercase (compatibility with technical_indicators_lib)
    df.columns = map(str.lower, df.columns)
    
    return df

___

## Add Technical Indicators to df


#### Moving Averages
- includes several commonly used ranges and renames columns appropriately

In [ ]:
def calc_moving_averages(df):
    '''
    Adds select moving averages as columns to the dataframe
    
    Args:
        df(pandas.DataFrame): OHLC dataframe retrieved from yahoofinanace api
        
    Returns:
        pandas.DataFrame: modified pandas dataframe with select moving averages
    '''
    
    # suppress pandas copy warning
    df.is_copy = False
    
    # calculate select moving averages
    d = [5, 8, 10, 13, 20, 21, 34, 50, 55, 89, 100]
    for day in d:
        df = sma.get_value_df(df, time_period=day)
        col_name = 'MA_' + str(day) + 'd'
        df.rename(columns = {'SMA': col_name}, inplace=True)
        
    # calculate exponential moving averages
    d_ema = [5, 12, 20, 26, 50, 200]
    for day in d_ema:
        df = ema.get_value_df(df, time_period=day)
        col_name = 'EMA_' + str(day) + 'd'
        df.rename(columns = {'EMA': col_name}, inplace=True)

    return df

#### Other indicators
- function to add any indicator from the library to the df

In [ ]:
def calc_indicator(df, indicator):
    '''
    Calculates the given technical indicator and appends the df with its values
    
    Args:
        df (pandas.DataFrame): OHLC dataframe
        indicator (function): function from tecnical_indicators_lib
        
    Returns:
        pandas.DataFrame: modified dataframe with the specified indicator
    '''
    
    # suppress pandas copy warning
    df.is_copy = False
    
    # get indicator value with dataframe method
    df = indicator.get_value_df(df)
    
    return df

In [ ]:
indicators_list = [adi, atr, bb, cci, cmf, cho, chv, dc, dpo, emv, ema, kc, kst, mi, mem, mom, macd, nvi, obv, 
                   pvt, roc, rocv, rsi, sma, vlt, stochastic_k_and_d, trix, tr, tsi, typ, vhf, wcl]

# fi, mfi, pc, po, vo

In [ ]:
print(fi.info())

In [ ]:
for indicator in indicators_list:
    calc_indicator(QQQ, indicator)

In [ ]:
QQQ.head()

### Candle Charts

In [ ]:
def candle_plot(df):
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                                        open=df['open'],
                                        high=df['high'],
                                        low=df['low'],
                                        close=df['close'])])
    fig.show()

In [12]:
BTC.head()

,open,high,low,close,volume,dividends,stock splits
Date,,,,,,,
2018-01-01,14112.200195,14112.200195,13154.700195,13657.200195,10291200000,0,0
2018-01-02,13625.000000,15444.599609,13163.599609,14982.099609,16846600192,0,0
2018-01-03,14978.200195,15572.799805,14844.500000,15201.000000,16871900160,0,0
2018-01-04,15270.700195,15739.700195,14522.200195,15599.200195,21783199744,0,0
2018-01-05,15477.200195,17705.199219,15202.799805,17429.500000,23840899072,0,0


In [ ]:
candle_plot(BTC)